<a href="https://colab.research.google.com/github/lavanyabollepalli25/GEN-AI/blob/main/bert_predicts_mlm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# Import required libraries
from transformers import AutoTokenizer, AutoModelForMaskedLM
from scipy.special import softmax
import numpy as np

In [2]:
# Specify the pre-trained model to use: BERT-base-cased
model_name = "bert-base-cased"

# Instantiate the tokenizer and model for the specified pre-trained model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
model

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [3]:
# Get the mask token from the tokenizer
mask = tokenizer.mask_token
print(mask)

[MASK]


In [14]:
# Create a sentence with a mask token to be filled in by the model
sentence = f"I need {mask} dress for my birthday."
# Tokenize the sentence
tokens = tokenizer.tokenize(sentence)
print(tokens)

['I', 'need', '[MASK]', 'dress', 'for', 'my', 'birthday', '.']


In [16]:
# Encode the sentence using the tokenizer and return the input tensors
encoded_inputs = tokenizer(sentence, return_tensors='pt')
print(encoded_inputs)

{'input_ids': tensor([[ 101,  146, 1444,  103, 3642, 1111, 1139, 5913,  119,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [17]:
# Get the model's output for the input tensors
outputs = model(**encoded_inputs)
print(outputs)

MaskedLMOutput(loss=None, logits=tensor([[[ -7.4047,  -7.3128,  -7.4839,  ...,  -6.3032,  -5.9746,  -6.3536],
         [ -7.9847,  -8.2545,  -8.0368,  ...,  -6.7119,  -6.5203,  -6.9465],
         [ -9.4555,  -9.5114,  -9.7188,  ...,  -6.5209,  -5.3159,  -6.1432],
         ...,
         [ -9.4146,  -9.0938,  -8.4870,  ...,  -5.5974,  -4.5083,  -6.9645],
         [-12.2302, -12.2268, -12.2036,  ...,  -9.4412,  -9.3976, -10.4910],
         [-15.0810, -14.8745, -14.7900,  ..., -11.4825, -11.9654, -12.4503]]],
       grad_fn=<ViewBackward0>), hidden_states=None, attentions=None)


In [18]:
# Detach the logits from the model's output and convert them to numpy arrays
logits = outputs.logits.detach().numpy()[0]
logits.shape

(10, 28996)

In [19]:
len(tokens)

8

In [20]:
# Extract the logits for the mask token
mask_logits = logits[tokens.index(mask) + 1]
print(mask_logits)

[-8.771986  -7.7710304 -8.222405  ... -6.5756626 -5.579332  -7.170435 ]


In [25]:
mask_logits.shape

(28996,)

In [21]:
# Calculate the confidence scores for each possible token using softmax
confidence_scores = softmax(mask_logits)
print(confidence_scores)

[2.33502453e-11 6.35332342e-11 4.04548686e-11 ... 2.09962908e-10
 5.68647740e-10 1.15834044e-10]


In [22]:
confidence_scores.sum()

1.0000001

In [26]:
# Print the top 5 predicted tokens and their confidence scores
for i in np.argsort(confidence_scores)[::-1][:10]:
    pred_token = tokenizer.decode(i)
    score = confidence_scores[i]

    # Print the predicted sentence with the mask token replaced by the predicted token, and the confidence score
    print(sentence.replace(mask, pred_token), score)

I need to dress for my birthday. 0.48066893
I need a dress for my birthday. 0.39742133
I need that dress for my birthday. 0.034301654
I need this dress for my birthday. 0.023897026
I need the dress for my birthday. 0.020842321
I need my dress for my birthday. 0.019022083
I need another dress for my birthday. 0.010346538
I need your dress for my birthday. 0.0042017344
I need some dress for my birthday. 0.002170006
I need her dress for my birthday. 0.0014511945
